In [1]:
import os
import shutil
import dataiku
import pandas as pd
import mlflow
from sklearn.linear_model import LogisticRegression

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  """


In [2]:
from dss_mlflow import preprocessing

In [3]:
client = dataiku.api_client()
project = client.get_default_project()

In [4]:
# get train dataset
train_dataset = dataiku.Dataset("training_data")
evaluation_dataset = dataiku.Dataset("eval_data")

In [5]:
# get output saved model
sm = project.get_saved_model("VdHxdbkg")

# get train dataset as a pandas dataframe
df = train_dataset.get_dataframe()

In [6]:
# get the path of a local managed folder where to temporarily save the trained model
mf = dataiku.Folder("cTcsiBqc")
path = mf.get_path()

model_subdir = "my_subdir"
model_dir = os.path.join(path, model_subdir)

In [7]:
X, y = preprocessing.clean_df(df)

In [9]:
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

try:
    # ...train your model...
    clf = LogisticRegression(random_state=0).fit(X, y)

    # ...save it with package specific MLflow method (here, SKlearn)...
    mlflow.sklearn.save_model(clf, model_dir)

    # import the model, creating a new version
    mlflow_version = sm.import_mlflow_version_from_managed_folder("v03", "cTcsiBqc", model_subdir, "py36_mlflow")
finally:
    shutil.rmtree(model_dir)

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
# setting metadata (target name, classes,...)
mlflow_version.set_core_metadata(target_column_name="y",
                             class_labels=["no", "yes"],
                             get_features_from_dataset="eval_data_prepared")

In [0]:
# evaluate the performance of this new version, to populate the performance screens of the saved model version in DSS
mlflow_version.evaluate("eval_data_prepared")